In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

# Clean Headers
school_data_complete = school_data_complete.rename(columns = {'student_name': 'Student Name', 'school_name':'School Name', 
                        'reading_score':'Reading Score', 'maths_score':'Maths Score', 'type':'School Type', 'size':'Total Students', 'budget': 'Total School Budget'})

school_data_complete.head()

,Student ID,Student Name,gender,year,School Name,Reading Score,Maths Score,School ID,School Type,Total Students,Total School Budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [2]:
#Total Number of Schools
totSchools = (school_data_complete['School Name'].nunique())

#Total Number of Students
totStudents = int(school_data_complete['Student ID'].nunique())

#Total overall budget across all schools
totBudget = school_data_complete['Total School Budget'].unique().sum()
totBudget

#The average Math score
totAveMath = round(school_data_complete['Maths Score'].mean(), 2)
totAveMath 

#The average Reading score
totAveRead = round(school_data_complete['Reading Score'].mean(), 2)
totAveRead 

#Students who passed Maths across all schools
stuMath50_df = school_data_complete.loc[ school_data_complete["Maths Score"] >= 50,:]
stuMath50count = stuMath50_df["Student ID"].count()
perMath50 = round((stuMath50count/totStudents)*100,2)

#Students who passed Reading across all schools
stuReading50_df = school_data_complete.loc[ school_data_complete["Reading Score"] >= 50,:]
stuReading50count = stuReading50_df["Student ID"].count()
perReading50 = round((stuReading50count/totStudents)*100,2)

#Students who passed both Maths and Reading across all schools
stuMR50_df = school_data_complete.loc[(school_data_complete["Reading Score"] >= 50) & 
                                      (school_data_complete["Maths Score"] >= 50),:]
stuMR50count = stuMR50_df["Student ID"].count()
perMR50 = round((stuMR50count/totStudents)*100,2)

#Storing relevant information in DataFrame
LGAS = pd.DataFrame({'Total Schools':[totSchools],'Total Students': [totStudents],
                      'Total Budget':[totBudget], 'Average Maths Score':[totAveMath],
                      'Average Reading Score':[totAveRead], '% Passing Maths':[perMath50],
                       '% Passing Reading':[perReading50], '% Overall Passing':[perMR50]})

#Formatting Output
LGAS["Total Budget"] = LGAS["Total Budget"].astype(float).map("${:,.2f}".format)
LGAS["Total Students"] = LGAS["Total Students"].astype(float).map("{:,.0f}".format)

#Display Local Government Area Summery
print("-----------------------------")
print("Local Government Area Summary")
print("-----------------------------")
print(LGAS)

-----------------------------
Local Government Area Summary
-----------------------------
   Total Schools Total Students    Total Budget  Average Maths Score  \
0             15         39,170  $24,649,428.00                70.34   

   Average Reading Score  % Passing Maths  % Passing Reading  \
0                  69.98            86.08              84.43   

   % Overall Passing  
0              72.81  


#Starting to create the School Summary DataFrame (table)

In [3]:
#Creating Average Reading Scores by School
aveRead = pd.DataFrame(school_data_complete.groupby(["School Name"])["Reading Score"].mean())
aveRead["Average Reading Score"] = aveRead["Reading Score"]
aveRead = aveRead[["Average Reading Score"]].sort_values("Average Reading Score", ascending=False)
aveRead.head()

,Average Reading Score
School Name,
Holden High School,71.660422
Pena High School,71.613306
Cabrera High School,71.359526
Griffin High School,71.245232
Bailey High School,71.008842


In [4]:
#Creating Average Math Scores by School
aveMath = pd.DataFrame(school_data_complete.groupby(["School Name"])["Maths Score"].mean())
aveMath = aveMath.rename(columns = {"Maths Score":"Average Maths Score"})
aveMath = aveMath[["Average Maths Score"]].sort_values("Average Maths Score", ascending=False)
aveMath.head()

,Average Maths Score
School Name,
Holden High School,72.583138
Bailey High School,72.352894
Pena High School,72.088358
Rodriguez High School,72.047762
Wright High School,72.047222


In [5]:
#Creating Passed Math by School (total student numbers)
passMath = pd.DataFrame(school_data_complete.loc[school_data_complete["Maths Score"] >= 50,:])
passMath = pd.DataFrame(passMath.groupby("School Name").count())
passMath = passMath.rename(columns = {"Maths Score":"Students Passing Math"})
passMath = passMath[["Students Passing Math"]].sort_values("Students Passing Math", ascending=False)
passMath.head()

,Students Passing Math
School Name,
Bailey High School,4560
Johnson High School,3907
Hernandez High School,3752
Rodriguez High School,3631
Figueroa High School,2408


In [6]:
#Creating Passed Reading by School (total student numbers)
passRead = pd.DataFrame(school_data_complete.loc[school_data_complete["Reading Score"] >= 50,:])
passRead = pd.DataFrame(passRead.groupby("School Name").count())
passRead = passRead.rename(columns = {"Reading Score":"Students Passing Reading"})
passRead = pd.DataFrame(passRead[["Students Passing Reading"]]).sort_values("Students Passing Reading", ascending=False)
passRead.head()

,Students Passing Reading
School Name,
Bailey High School,4348
Johnson High School,3903
Hernandez High School,3795
Rodriguez High School,3495
Figueroa High School,2442


In [7]:
#Creating Passed Math and Reading by School (total student numbers)
passAll = pd.DataFrame(school_data_complete.loc[(school_data_complete["Reading Score"] >= 50)
                                                & (school_data_complete["Maths Score"] >= 50),:])
passAll  = pd.DataFrame(passAll.groupby("School Name").count())
passAll = passAll.rename(columns = {"Student ID":"Students Passing Overall"})
passAll = passAll[["Students Passing Overall"]].sort_values("Students Passing Overall", ascending=False)
passAll.head()

,Students Passing Overall
School Name,
Bailey High School,3985
Johnson High School,3199
Rodriguez High School,3176
Hernandez High School,3076
Figueroa High School,1995


In [8]:
#Forming the main School Summary table - pulling in columns from school_data_complete DF - and dropping any duplicate rows with relation to "School Name"
#This should provide a dataframe where by we only have a single row per School Name
Summary = school_data_complete[["School Name","School Type",
                                "Total Students","Total School Budget"]
                              ].drop_duplicates(["School Name"])

#Calculate the Per Student Budget using the values within the relevant columns
Summary["Per Student Budget"] = Summary["Total School Budget"]/Summary["Total Students"]
Summary.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget
0,Huang High School,Government,2917,1910635,655.0
2917,Figueroa High School,Government,2949,1884411,639.0
5866,Shelton High School,Independent,1761,1056600,600.0
7627,Hernandez High School,Government,4635,3022020,652.0
12262,Griffin High School,Independent,1468,917500,625.0


In [9]:
#Merging all the calculated dataframes with our Summary dataframe.
Summary = Summary.merge(aveRead,on='School Name').merge(aveMath,on='School Name').merge(passMath, on="School Name").merge(passRead,on="School Name").merge(passAll,on="School Name")

#Column Title Renaming: 
Summary = Summary.rename(columns = {"Students Passing Math":"% Passing Maths"})
Summary = Summary.rename(columns = {"Students Passing Reading":"% Passing Reading"})
Summary = Summary.rename(columns = {"Students Passing Overall":"% Overall Passing"})

#Calculations:
Summary["% Passing Maths"] = (Summary["% Passing Maths"]/Summary["Total Students"])*100
Summary["% Passing Reading"] = (Summary["% Passing Reading"]/Summary["Total Students"])*100
Summary["% Overall Passing"] = (Summary["% Overall Passing"]/Summary["Total Students"])*100

#Created a duplicate Summary_raw - Pre 'Value Formatting' Copy of Summary (DataFrame)
Summary_raw = Summary.copy(deep=False)

#Formatting Value Appearance:
Summary["Total School Budget"] = Summary["Total School Budget"].astype(float).map("${:,.2f}".format)
Summary["Per Student Budget"] = Summary["Per Student Budget"].astype(float).map("${:,.2f}".format)

#Sort by School Name  
Summary = Summary.sort_values("School Name")

#Set Index to School Name
Summary.set_index("School Name")

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Maths Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,71.008842,72.352894,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.359526,71.657158,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,69.077993,68.698542,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.572472,69.091274,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.245232,71.788147,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,69.186408,68.874865,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,71.660422,72.583138,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.910525,68.935207,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,69.039277,68.843100,82.062592,81.978576,67.191766


In [10]:
#Displaying the top 5 schools based on their overall passing percentage
TopSchoolOverall = Summary.sort_values("% Overall Passing", ascending=False).set_index("School Name")
TopSchoolOverall.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Maths Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.245232,71.788147,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.359526,71.657158,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,71.008842,72.352894,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,70.969444,72.047222,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,70.935984,72.047762,90.797699,87.396849,79.419855


In [11]:
#Displaying the bottom 5 schools based on their overall passing percentage
BotSchoolOverall = Summary.sort_values("% Overall Passing", ascending=True).set_index("School Name")
BotSchoolOverall.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Maths Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,69.186408,68.874865,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.910525,68.935207,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,69.039277,68.843100,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,68.876916,69.170828,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.572472,69.091274,82.438846,82.219788,67.469880


In [12]:
#Creating a dataframe to store the average of the Year 9s across each school
math9 = pd.DataFrame(school_data_complete.loc[school_data_complete["year"] == 9,
                                                                  ["School Name", "Maths Score"]]
                    ).groupby("School Name").mean()
math9 = math9.rename(columns = {"Maths Score":"Year 9"})
math9 = math9[["Year 9"]].sort_values("Year 9", ascending=False)
math9.head()

,Year 9
School Name,
Shelton High School,72.932075
Griffin High School,72.789731
Bailey High School,72.493827
Cabrera High School,72.321970
Pena High School,71.996364


In [13]:
#Creating a dataframe to store the average of the Year 10s across each school
math10 = pd.DataFrame(school_data_complete.loc[school_data_complete["year"] == 10,
                                                                   ["School Name", "Maths Score"]]
                     ).groupby("School Name").mean()
math10 = math10.rename(columns = {"Maths Score":"Year 10"})
math10 = math10[["Year 10"]].sort_values("Year 10", ascending=False)
math10.head()

,Year 10
School Name,
Holden High School,75.105263
Shelton High School,72.506696
Cabrera High School,72.437768
Pena High School,72.396000
Wright High School,72.179012


In [14]:
#Creating a dataframe to store the average of the Year 11s across each school
math11 = pd.DataFrame(school_data_complete.loc[school_data_complete["year"] == 11,
                                                                    ["School Name", "Maths Score"]]
                     ).groupby("School Name").mean()
math11["Year 11"] = math11.rename(columns = {"Maths Score":"Year 11"})
math11 = math11[["Year 11"]].sort_values("Year 11", ascending=False)
math11.head()

,Year 11
School Name,
Wright High School,73.275862
Pena High School,72.523438
Bailey High School,72.374900
Rodriguez High School,72.364811
Griffin High School,71.692521


In [15]:
#Creating a dataframe to store the average of the Year 12s across each school
math12 = pd.DataFrame(school_data_complete.loc[school_data_complete["year"] == 12,
                                                                   ["School Name", "Maths Score"]]
                      ).groupby("School Name").mean()
math12["Year 12"] = math12.rename(columns = {"Maths Score":"Year 12"})
math12 = math12[["Year 12"]].sort_values("Year 12", ascending=False)
math12.head()

,Year 12
School Name,
Holden High School,73.409639
Bailey High School,72.675097
Shelton High School,72.331536
Rodriguez High School,72.154626
Griffin High School,71.469178


In [16]:
#Merging all the dataframes together to make our Maths Average by School Summary Table
mathAveByYear = math9.merge(math10,on="School Name").merge(math11,on="School Name").merge(math12,on="School Name")
mathAveByYear.sort_values("School Name").head()

,Year 9,Year 10,Year 11,Year 12
School Name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178


In [17]:
#Creating a dataframe to store the average of the Year 9s across each school
read9 = pd.DataFrame(school_data_complete.loc[school_data_complete["year"] == 9,
                                                                  ["School Name", "Reading Score"]])
read9 = pd.DataFrame(read9.groupby("School Name").mean())
read9 = read9.rename(columns = {"Reading Score":"Year 9"})
read9 = read9[["Year 9"]].sort_values("Year 9", ascending=False)
read9.head()

,Year 9
School Name,
Griffin High School,72.026895
Wright High School,71.823529
Holden High School,71.598425
Cabrera High School,71.172348
Pena High School,70.949091


In [18]:
#Creating a dataframe to store the average of the Year 10s across each school
read10 = pd.DataFrame(school_data_complete.loc[school_data_complete["year"] == 10,
                                                                   ["School Name", "Reading Score"]])
read10 = pd.DataFrame(read10.groupby("School Name").mean())
read10 = read10.rename(columns = {"Reading Score":"Year 10"})
read10 = read10[["Year 10"]].sort_values("Year 10", ascending=False)
read10.head()

,Year 10
School Name,
Pena High School,72.324000
Wright High School,71.386831
Cabrera High School,71.328326
Holden High School,71.096491
Bailey High School,70.848265


In [19]:
#Creating a dataframe to store the average of the Year 11s across each school
read11 = pd.DataFrame(school_data_complete.loc[school_data_complete["year"] == 11,
                                                                   ["School Name", "Reading Score"]])
read11 = pd.DataFrame(read11.groupby("School Name").mean())
read11 = read11.rename(columns = {"Reading Score":"Year 11"})
read11 = read11[["Year 11"]].sort_values("Year 11", ascending=False)
read11.head()

,Year 11
School Name,
Holden High School,73.310680
Griffin High School,72.385042
Pena High School,71.703125
Rodriguez High School,71.424453
Cabrera High School,71.201245


In [20]:
#Creating a dataframe to store the average of the Year 12s across each school
read12 = pd.DataFrame(school_data_complete.loc[school_data_complete["year"] == 12,
                                                                   ["School Name", "Reading Score"]]
                     ).groupby("School Name").mean()
read12 = read12.rename(columns = {"Reading Score":"Year 12"})
read12 = read12[["Year 12"]].sort_values("Year 12", ascending=False)
read12.head()

,Year 12
School Name,
Bailey High School,72.195525
Cabrera High School,71.856021
Pena High School,71.513812
Rodriguez High School,71.414449
Holden High School,70.481928


In [21]:
#Merging all the dataframes together to make our Reading Average by School Summary Table
readAveByYear = pd.merge(read9, read10, on="School Name").merge(read11, on="School Name").merge(read12, on="School Name")
readAveByYear.sort_values("School Name").head()

,Year 9,Year 10,Year 11,Year 12
School Name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932


In [22]:
#Created bins for School Spending per Student
spending_bin = [0,585,630,645,680]
spending_labels = ["<$585", "$585-630","$630-645","$645-680"] 

In [23]:
#Create a new Summary table column for our bins to apply their relevant labels to - We reference the Summary_raw df as it has unformatted numerical data. 
Summary["Spending Ranges (Per Student)"] = pd.cut(Summary_raw["Per Student Budget"], spending_bin, labels=spending_labels)
spendingRangeGroup = Summary.groupby("Spending Ranges (Per Student)")
spendingRangeGroup = round(spendingRangeGroup[["Average Maths Score","Average Reading Score",
                          "% Passing Maths","% Passing Reading","% Overall Passing"]].mean(),2)
spendingRangeGroup

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


In [24]:
#Created bins for School Size 
size_bin = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)","Large (2000-5000)"] 

In [25]:
#Created new Summary table column to store our bins to apply their relevant labels to
Summary["School Size"] = pd.cut(Summary["Total Students"], size_bin, labels=size_labels)
sizeGroup = Summary.groupby("School Size")
sizeGroup= round(sizeGroup[["Average Maths Score","Average Reading Score","% Passing Maths",
                 "% Passing Reading","% Overall Passing"]].mean(),2)
sizeGroup


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.64,90.81,87.56,79.07
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


In [27]:
schooltypeGroup = Summary.groupby("School Type")
schooltypeGroup = round(schooltypeGroup[["Average Maths Score","Average Reading Score",
                 "% Passing Maths","% Passing Reading","% Overall Passing"]].mean(),2)
schooltypeGroup

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.83,69.68,84.46,83.59,70.70
Independent,71.37,70.72,89.20,86.25,76.97
